In [1]:
import tensorflow as tf

In [2]:
import pandas as pd

In [4]:
data = pd.read_csv(r"\\vector.e3b.columbia.edu\e3b\Globus Database\UWIN Tagged Photos\batch_002\detections_to_update_2024-02-07.csv")

In [5]:
data["is_present"] = data['species'].apply(lambda x: "not_present" if pd.isna(x) else "present")

In [6]:
data.head(4)

,Unnamed: 0,photoName,timestamp,speciesID,species,count,siteName,siteAbbrev,longitude,latitude,validationStatus,group,updateCommonName,updateNumIndividuals,is_present
0,15000,VID31205-01147.jpg,2022-07-16 21:55:05,7,Bird,1,Brower Park,BrPk,-73.942589,40.673672,1,2,NaN,NaN,present
1,15001,VID31205-01148.jpg,2022-07-16 21:58:04,7,Bird,1,Brower Park,BrPk,-73.942589,40.673672,1,2,NaN,NaN,present
2,15002,VID31205-01149.jpg,2022-07-16 21:58:41,7,Bird,1,Brower Park,BrPk,-73.942589,40.673672,1,2,NaN,NaN,present
3,15003,VID31205-01150.jpg,2022-07-16 21:59:22,7,Bird,1,Brower Park,BrPk,-73.942589,40.673672,1,2,NaN,NaN,present


In [9]:
import os
from details import folder
directory = os.path.join(folder, r"batch_002")

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
datagen = ImageDataGenerator(rescale=1./255.)

In [12]:
test_data = datagen.flow_from_dataframe(dataframe=data, 
                                            directory=directory, 
                                            x_col='photoName', 
                                            y_col='is_present', 
                                            target_size=(224,224), 
                                            class_mode='binary', 
                                            batch_size=32)

Found 15000 validated image filenames belonging to 2 classes.


In [17]:
from keras.models import load_model

In [20]:
from keras.applications import EfficientNetB4

In [21]:
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
out = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.models.Model(inputs = base_model.input, outputs = out)

In [22]:
model.load_weights(r"C:\Users\ab5640\Desktop\Python\Project\best_weights_acc_effb4\best_weights_acc.h5")

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']      

In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.evaluate(test_data)

469/469 [==============================] - 5030s 11s/step - loss: 0.6321 - accuracy: 0.6759


[0.632111132144928, 0.6758666634559631]

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True